**Установка библиотек**

**Импорт библиотек**

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

**Загрузка датасета**


In [2]:

df = pd.read_csv('rusentitweet_full.csv')  # Замени на реальное имя файла
texts = df['text'].astype(str).tolist()  # Колонка с текстами твитов


**Объединяем все тексты для character-level (или token-level)**

In [3]:
full_text = '\n'.join(texts).lower()  # Нижний регистр для простоты

**Character-level подготовка**

In [6]:
chars = sorted(list(set(full_text)))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

max_len = 60  # Длина последовательности
step = 10
sequences = []
next_chars = []

for i in range(0, len(full_text) - max_len, step):
    sequences.append(full_text[i: i + max_len])
    next_chars.append(full_text[i + max_len])

X = np.zeros((len(sequences), max_len, vocab_size), dtype=bool)
y = np.zeros((len(sequences), vocab_size), dtype=bool)

for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        X[i, t, char_to_ix[char]] = 1
    y[i, char_to_ix[next_chars[i]]] = 1

**Модель LSTM**

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(max_len, vocab_size), return_sequences=True))
model.add(LSTM(256))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, batch_size=128, epochs=30)  # Больше эпох для лучшего результата

Epoch 1/30

632/632 [==============================] - 314s 491ms/step - loss: 3.7761
Epoch 2/30
632/632 [==============================] - 319s 504ms/step - loss: 3.0253
Epoch 3/30
632/632 [==============================] - 307s 486ms/step - loss: 2.7973
Epoch 4/30
632/632 [==============================] - 316s 500ms/step - loss: 2.7132
Epoch 5/30
632/632 [==============================] - 311s 492ms/step - loss: 2.6594
Epoch 6/30
632/632 [==============================] - 308s 487ms/step - loss: 2.6124
Epoch 7/30
632/632 [==============================] - 314s 497ms/step - loss: 2.5739
Epoch 8/30
632/632 [==============================] - 312s 494ms/step - loss: 2.5336
Epoch 9/30
632/632 [==============================] - 304s 482ms/step - loss: 2.4982
Epoch 10/30
632/632 [==============================] - 305s 483ms/step - loss: 2.4681
Epoch 11/30
632/632 [==============================] - 309s 488ms/step - loss: 2.4369
Epoch 12/30
632/632 [==============================] - 308s 48

**Генерация поста**

In [8]:
def generate_post(seed_text, length=200):
    generated = seed_text
    seed_text = seed_text.lower()
    for _ in range(length):
        x = np.zeros((1, max_len, vocab_size))
        for t, char in enumerate(seed_text):
            if t < max_len:
                x[0, t, char_to_ix.get(char, 0)] = 1
        preds = model.predict(x, verbose=0)[0]
        next_ix = np.argmax(preds)
        next_char = ix_to_char[next_ix]
        generated += next_char
        seed_text = seed_text[1:] + next_char
    return generated